## Data Snippets

In [73]:
import geopandas as gpd
import numpy as np
import pandas as pd
import shared_utils
from calitp_data_analysis.sql import to_snakecase

In [32]:
from calitp_data_analysis.tables import tbls
from calitp_data_analysis.sql import query_sql
from siuba import *

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Exercise 2

In [5]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/bus_service_increase/"
FILE_NAME = "ntd_metrics_2019.csv"

metrics = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME}")

In [40]:
metrics.to_parquet("./data/exercise_2_3_ntd_metrics_2019.parquet")

In [7]:
FILE_NAME = "ntd_vehicles_2019.csv"
vehicles = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME}")


In [41]:
vehicles.to_parquet("./data/exercise_2_ntd_vehicles_2019.parquet")

### Exercise 4

In [64]:
FEEDS = [
    "25c6505166c01099b2f6f2de173e20b9", # Caltrain
    "52639f09eb535f75b33d2c6a654cb89e", # Merced
    "e1d7185ffb6f73f7d373787910f0bf30" # Lodi
]

In [65]:
stops = (
    tbls.mart_gtfs.dim_stops()
    >> filter(_.feed_key.isin(FEEDS))
    >> select(_.feed_key, _.stop_id, 
             _.stop_lat, _.stop_lon, _.stop_name)
    >> arrange(_.feed_key, _.stop_id, 
               _.stop_lat, _.stop_lon)
    >> collect() 
)

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [66]:
len(stops) * .4

320.8

In [67]:
stops2 = stops.sample(250)

In [69]:
stops2.feed_key.value_counts()

52639f09eb535f75b33d2c6a654cb89e    147
e1d7185ffb6f73f7d373787910f0bf30     68
25c6505166c01099b2f6f2de173e20b9     23
ba587fd71b4f9a956ff36e309e6acf3f     12
Name: feed_key, dtype: int64

In [71]:
stops2.to_parquet("./data/exercise_4_stops_sample.parquet")

### Exercise 5

In [74]:
feeds_to_names = shared_utils.gtfs_utils_v2.schedule_daily_feed_to_organization(
    selected_date = "2022-06-01",
    get_df = True
)[["feed_key", "name"]].drop_duplicates()

In [75]:
feeds_to_names.shape

(200, 2)

In [76]:
OPERATORS = [
    "Alhambra Schedule", 
    "San Diego Schedule",
    "Big Blue Bus Schedule",
]

SUBSET_FEEDS = feeds_to_names[
    feeds_to_names.name.isin(OPERATORS)
].feed_key.tolist()

In [78]:
stops_ex5 = (
    tbls.mart_gtfs.fct_daily_scheduled_stops()
    >> filter(_.feed_key.isin(SUBSET_FEEDS))
    >> select(_.feed_key, 
              _.stop_id, _.pt_geom)
    >> collect()
)

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 500 Query exceeded limit for bytes billed: 5000000000. 7122976768 or higher required.

Location: us-west2
Job ID: 4cba6648-bc57-40c2-b95b-e8eb0abd06ad

[SQL: SELECT `mart_gtfs.fct_daily_scheduled_stops_1`.`feed_key`, `mart_gtfs.fct_daily_scheduled_stops_1`.`stop_id`, `mart_gtfs.fct_daily_scheduled_stops_1`.`pt_geom` 
FROM `mart_gtfs.fct_daily_scheduled_stops` AS `mart_gtfs.fct_daily_scheduled_stops_1` 
WHERE `mart_gtfs.fct_daily_scheduled_stops_1`.`feed_key` IN UNNEST(%(feed_key_1:STRING)s)]
[parameters: {'feed_key_1': ['239e56d11510f71d7182a24c5621be8c', '455fadac7ed63a72e7d3f36273d78313', 'd76560b3dfecce2d588023bf1d1c4c2d']}]
(Background on this error at: https://sqlalche.me/e/14/4xp6)